In [1]:
from code.evaluation import evaluate_all_models, evaluate_all_reductions, evaluate_model, evaluate_reduction

In [2]:
dataset_name = 'grid'
method = 'knn'
metric = 'Accuracy'
    
evaluate_model(dataset_name, method, metric, type_evaluation='autorank', plot=False)

Best Hyperparameters:
{'K': 7, 'Distance': 'minkowski2', 'Voting scheme': 'Majority_class', 'Weight scheme': 'Mutual_classifier'}
Best Accuracy: 0.9672
The statistical analysis was conducted for 7 populations with 10 paired samples.
The family-wise significance level of the tests is alpha=0.050.
We rejected the null hypothesis that the population is normal for the population Mj_MC_m2_7 (p=0.002). Therefore, we assume that not all populations are normal.
Because we have more than two populations and the populations and one of them is not normal, we use the non-parametric Friedman test as omnibus test to determine if there are any significant differences between the median values of the populations. We use the post-hoc Nemenyi test to infer which differences are significant. We report the median (MD), the median absolute deviation (MAD) and the mean rank (MR) among all populations over the samples. Differences between populations are significant, if the difference of the mean rank is gre

In [10]:
evaluate_all_models(type_evaluation='autorank', plot=False)

Evaluating method knn on dataset grid and metric Accuracy
Best Hyperparameters:
{'K': 7, 'Distance': 'minkowski2', 'Voting scheme': 'Majority_class', 'Weight scheme': 'Mutual_classifier'}
Best Accuracy: 0.9672
The statistical analysis was conducted for 7 populations with 10 paired samples.
The family-wise significance level of the tests is alpha=0.050.
We rejected the null hypothesis that the population is normal for the population Mj_MC_m2_7 (p=0.002). Therefore, we assume that not all populations are normal.
Because we have more than two populations and the populations and one of them is not normal, we use the non-parametric Friedman test as omnibus test to determine if there are any significant differences between the median values of the populations. We use the post-hoc Nemenyi test to infer which differences are significant. We report the median (MD), the median absolute deviation (MAD) and the mean rank (MR) among all populations over the samples. Differences between populations 

In [4]:
dataset_name = 'grid'
method = 'knn'
metric = 'Accuracy'
reduction_method = 'CNN'

In [11]:
evaluate_reduction(dataset_name, method, reduction_method, type_evaluation='our_criteria', summary_statistics=None,plot=False)

Evaluating Accuracy
Paired t-test statistic: 9.179625086610036, p-value: 0.05
Significant difference found between the two models.
Evaluating Solving Time
Paired t-test statistic: 20.252388234999486, p-value: 0.05
Significant difference found between the two models.


In [12]:
summary_statistics = evaluate_all_reductions(type_evaluation='our_criteria', plot=False)

Evaluating method knn on dataset grid and reduction CNN
Evaluating Accuracy
Paired t-test statistic: 9.179625086610036, p-value: 0.05
Significant difference found between the two models.
Evaluating Solving Time
Paired t-test statistic: 20.252388234999486, p-value: 0.05
Significant difference found between the two models.
Evaluating method knn on dataset grid and reduction DROP
Evaluating Accuracy
Paired t-test statistic: 4.597706657399072, p-value: 0.05
Significant difference found between the two models.
Evaluating Solving Time
Paired t-test statistic: 26.8321003092542, p-value: 0.05
Significant difference found between the two models.
Evaluating method knn on dataset grid and reduction EENTh
Evaluating Accuracy
Paired t-test statistic: -0.2964804329914583, p-value: 0.05
No significant differences found between the two models.
Evaluating Solving Time
Paired t-test statistic: -1.8762901362857145, p-value: 0.05
No significant differences found between the two models.
Evaluating method s

In [13]:
summary_statistics

,Dataset,Method,Reduction,Accuracy,Solving Time,Num Samples
0,grid,knn,None,0.9672 ± 0.0122,3.2407 ± 0.3411,1700
1,grid,knn,CNN,0.6748 ± 0.0966,0.6907 ± 0.1936,225.2000 ± 5.8080
3,grid,knn,DROP,0.8000 ± 0.1207,0.7210 ± 0.1806,244.2000 ± 9.2952
5,grid,knn,EENTh,0.9682 ± 0.0128,4.5869 ± 2.0867,1526.7000 ± 6.5498
6,grid,svm,None,0.9142 ± 0.0155,0.2846 ± 0.0071,1700
7,grid,svm,CNN,0.8655 ± 0.0272,0.0358 ± 0.0115,225.2000 ± 5.8080
9,grid,svm,DROP,0.8797 ± 0.0278,0.0455 ± 0.0321,244.2000 ± 9.2952
11,grid,svm,EENTh,0.9142 ± 0.0161,0.2899 ± 0.1187,1526.7000 ± 6.5498
12,sick,knn,None,0.9626 ± 0.0087,47.9363 ± 0.8132,3395
13,sick,knn,CNN,0.9478 ± 0.0157,12.6354 ± 1.0498,585.0000 ± 22.5635
